## This was the code used to download the PDF's from the Secretary of State website

## We used selenium along with chrome to automate the process, while also using a virtual display to prevent the chrome browser from being displayed

In [140]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
import pickle
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

download_dir = '.' #Put location where you want to download the PDF's, default to current directory

chrome_options = Options()
chrome_options.add_experimental_option('prefs',  {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
    })
from pyvirtualdisplay import Display
display = Display(visible=0, size=(800, 600))
display.start()
driver = webdriver.Chrome('./chromedriver', options = chrome_options)  # Optional argument, if not specified will search path.

## The below cell opens the saved links of each company. We navigate to these links first and then navigate further to obtain the Annual Report PDF

In [125]:
f = open('saved_links.pkl','rb') #Put name of saved links pickle file
links = pickle.load(f)
f.close()

In [132]:
latest = 0 #Allows automatically restarting from the last completed link if the code fails due

## To combat the anti-scraper, we close and reopen the browser window every 100 links and also put sleep commands in a few places.

## The code can still fail sometimes, either when the downloaded PDF's take time to show up in the system or when the PDF link turns out faulty. Nothing can be done about these situations except skip over to the next company.

In [1]:
import os
driver.implicitly_wait(5)
start = latest
cur_count = 0
for link_n in range(start, len(links)):
    print(link_n)
    driver.get('http://lgwhd2o.x.incapdns.net/CorpWeb/CorpSearch/CorpSummary.aspx?sysvalue=' + links[link_n])
    time.sleep(1)
    try:
        el = driver.find_element_by_id('MainContent_btnViewFilings')
    except:
        driver.quit()
        driver = webdriver.Chrome('./chromedriver', options = chrome_options)
        driver.get('http://lgwhd2o.x.incapdns.net/CorpWeb/CorpSearch/CorpSummary.aspx?sysvalue=' + links[link_n])
        time.sleep(1)
    
    id_link = driver.find_element_by_id('MainContent_lblIDNumber').text.split('Identification Number: ')[1]
    filings_btn = driver.find_element_by_id('MainContent_btnViewFilings')
    filings_btn.click()

    try:
        el = driver.find_element_by_id('MainContent_grdSearchResults')
    except:
        url = driver.current_url
        driver.quit()
        driver = webdriver.Chrome('./chromedriver', options = chrome_options)
        driver.get(url)

    try:
        table = driver.find_element_by_id('MainContent_grdSearchResults')
    except:
        continue
    li = table.find_elements_by_tag_name('tr')
    ann_rep = None
    org_cert = None
    for i in li:
        if ann_rep != None:
            break
        if i.get_attribute('class') == 'GridHeader':
            continue
        if "Annual Report" in i.text and ann_rep == None:
            try:
                ann_rep = i.find_element_by_tag_name('a').get_attribute('href')
            except:
                break
    if ann_rep != None:
        driver.get(ann_rep)
        time.sleep(2)
        try:
            os.rename('./PDF/CorpSearchViewPDF.aspx','./PDF/' + id_link + '.pdf')
        except:
            driver.quit()
            driver = webdriver.Chrome('./chromedriver', options = chrome_options)
            time.sleep(1)
            driver.get(ann_rep)
            time.sleep(4)
            os.rename('./PDF/CorpSearchViewPDF.aspx','./PDF/' + id_link + '.pdf')
    latest = link_n
    cur_count += 1
    if cur_count == 100:
        driver.quit()
        driver = webdriver.Chrome('./chromedriver', options = chrome_options)
        cur_count = 0